<a href="https://colab.research.google.com/github/kw908/RL_doorkick/blob/main/Suttonbookexercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 3.18 \\
Relationship between $V^\pi$ and $q^\pi$:
$$
V^\pi(s) = \sum_{a \in A} \pi(a|s)q^\pi(s,a)
$$

In [1]:
## Exercise 3.14

import numpy as np

neighbors = [(0,0.25,2.3),
                   (0,0.25,0.4),
                   (0,0.25,-0.4),
                   (0,0.25,0.7)] # reward, probability, value(only accurate to one decimal place)
policy = np.array([0.25,0.25,0.25,0.25])
rewards = np.array([0,0,0,0])
middle_value = 0.7
gamma = 0.9
v_pi_middle = 0
state = [0,1]

for pa in policy:
    for v in neighbors:
       v_pi_middle += pa * v[1] * (v[0] + gamma*v[2])

print(v_pi_middle) #should evaluate to near 0.7


0.675


In [2]:
## Example 4.1

# evalucation on random policy
from Environments import gridworld_v0 as gw

env = gw.environment(sidelength=4) 
for s in env.states:
    print(s)

pi = {"up":[0.25,1.0], "down":[0.25,1.0], "left":[0.25,1.0], "right":[0.25,1.0]} #{action:[p(a), p(s',r|s,a)=1: (once the action is taken, it's 100% pr to get to one of the next states)]}
THETA = 0.1 #stop condition
GAMMA = 1.0 #non-discounted
terminal = False
episodes = 1 # number of episodes

# delta = 0
# V_s = 0
# q_s = 0
# v = 0

       

# # try evaluating first state
# while(delta<THETA):
#     v = V_s
#     for a in pi.keys:
#         terminal, next_state, r = gw.step(a)
#         if not terminal:
#           q_s += a.items[1] * (r + GAMMA*v[2])
#         else:
#           print(q_s)
        
       



TypeError: unhashable type: 'list'